<a href="https://colab.research.google.com/github/koklengyeo/aviation/blob/main/transport_v1_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREAMBLE

In [1]:
#FOR FUTURE REFERENCE
#DOWNGRADE PYTHON TO VERSION USED DURING DEVELOPMENT
#!apt-get update -y
#!apt-get install python3.10 python3.10-distutils
#!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!update-alternatives --config python3
#!apt-get install python3-pip
#!python3 -m pip install --upgrade pip --user

In [2]:
#FOR FUTURE REFERENCE
#DOWNGRADE PACKAGES TO VERSION USED DURING DEVELOPMENT
#!pip install IPython==7.34.0
#!pip install ipywidgets==7.7.1
#!pip install matplotlib==3.7.1
#!pip install numpy==1.25.2
#!pip install pandas==2.0.3

In [3]:
#IMPORT PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import *
from ipywidgets import *

# USER INPUT FORM (WIDGETS)

In [4]:
#DEFINE WIDGETS USED TO CAPTURE USER INPUTS

#TRANSPORT ACTIVITY
ta_widget=widgets.RadioButtons(
    value=None,
    options=["Freight","Passenger"],
    description="Type of Activity",
    disabled=False)

#TRANSPORT CATEGORY
tc_widget=widgets.RadioButtons(
    value=None,
    layout={"display":"none"},
    disabled=False)

#BASE YEAR
#MUST BE BETWEEN 2015 AND PRESENT YEAR
by_widget=widgets.BoundedIntText(
    value=datetime.now().year,
    min=2015,
    max=datetime.now().year,
    step=1,
    description="Base Year",
    disabled=False)

#TARGET YEAR
#MUST BE BETWEEN (PRESENT YEAR+5) AND 2050
ty_widget=widgets.BoundedIntText(
    value=datetime.now().year+5,
    min=datetime.now().year+5,
    max=2050,
    step=1,
    description="Target Year",
    disabled=False)

#WTW EMISSIONS IN TCO2E
#MUST BE >=0
wtw_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="WTW Emissions in tCO2e",
    disabled=False)

#TTW EMISSIONS IN TCO2E
#MUST BE >=0
wtt_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="WTT Emissions in tCO2e",
    disabled=False)

#WTT EMISSIONS IN TCO2E
#MUST BE >=0
ttw_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="TTW Emissions in tCO2e",
    disabled=False)

#BASE YEAR ACTIVITY IN TKM OR PKM
#MUST BE >=0
bya_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Activity",
    disabled=False)

#TARGET YEAR ACTIVITY IN TKM OR PKM
#MUST BE >=0
tya_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Activity",
    disabled=False)

#INVISIBLE BOOLEAN WIDGET TO REGISTER FIRST CLICK OF "CALCULATE TARGET" BUTTON
fc_widget=widgets.Valid(
    value=False,
    layout={"display":"none"})

In [5]:
#UPDATE WIDGETS DEPENDING ON TRANSPORT ACTIVITY & CATEGORY

def update_widgets(*args):
  if ta_widget.value=="Freight":
    #TC_WIDGET - UPDATE WITH FREIGHT TRANSPORT CATEGORIES
    tc_widget.options=[
        "2-3 Wheelers",
         "Light Commercial Vehicles",
          "Medium Freight Trucks",
           "Heavy Freight Trucks",
            "Both Medium & Heavy Freight Trucks",
             "Rail"]
    tc_widget.description="Freight Transport Category"
    tc_widget.layout={"width":"1100px"}
    tc_widget.style={"description_width":"600px"}
    #BYA_WIDGET & TYA_WIDGET - UPDATE UNITS TO TKM
    bya_widget.description="Activity in tkm"
    tya_widget.description="Activity in tkm"
  elif ta_widget.value=="Passenger":
    #TC_WIDGET - UPDATE WITH PASSENGER TRANSPORT CATEGORIES
    tc_widget.options=[
        "2-3 Wheelers",
         "Light Duty Vehicles",
          "Buses",
           "Mini-Buses",
            "Both Buses & Mini-Buses",
             "Urban Rail",
              "Non-Urban Rail",
               "Both Urban & Non-Urban Rail"]
    tc_widget.description="Passenger Transport Category"
    tc_widget.layout={"width":"1100px"}
    tc_widget.style={"description_width":"600px"}
    #BYA_WIDGET & TYA_WIDGET - UPDATE UNITS TO PKM
    bya_widget.description="Activity in pkm"
    tya_widget.description="Activity in pkm"
    #DISABLE WTT_WYDGET & TTW_WIDGET FOR URBAN RAIL
    if tc_widget.value=="Urban Rail":
      wtt_widget.disabled=True
      ttw_widget.disabled=True
      wtt_widget.value=0
      ttw_widget.value=0
    else:
      wtt_widget.disabled=False
      ttw_widget.disabled=False
ta_widget.observe(update_widgets,"value")
tc_widget.observe(update_widgets,"value")

In [6]:
#DEFINE WIDGETS USED TO DISPLAY TEXT

overall_header=widgets.HTML(
    value="<h1>TARGET SETTING FOR LAND TRANSPORT</h1>\
    <h3>Version 1.1.1 (Feb-23)</h3>This is intended to assist companies with \
    modelling targets on scope 3 transport related emissions. This target \
    model does not apply to scope 1 owned/controlled transport emissions. \
    Targets are aligned with a well below 2&deg;C scenario (IEA B2DS).")
basic_info_header=widgets.HTML(
    value="<h2>1. Basic Information</h2> Please indicate the type of transport \
    activity and the base year and target year. Target must cover a minimum of \
    5 years and a maximum of 10 years from the year it is submitted to the \
    SBTi for validation.")
by_emissions_header=widgets.HTML(
    value="<h2>2. Base Year Emissions</h2>Please provide, as far as available, \
    the well-to-wheel (WTW), well-to-tank (WTT), and tank-to-wheel (TTW) \
    emissions in the base year. The sum of WTT and TTW emissions must equate \
    to WTW emissions.")
by_activity_header=widgets.HTML(
    value="<h2>3. Base Year Activity</h2>Please provide the activity (in units \
    of tkm for freight activities and pkm for passenger activities) in the \
    base year.")
ty_activity_header=widgets.HTML(
    value="<h2>4. Target Year Activity</h2>Please provide the expected \
    activity in the target year.")
ty_activity_footer=widgets.HTML(
    value="Please ensure that all provided information is as complete and \
    accurate as possible. Click the button below to calculate the target.")
results_header=widgets.HTML(
    value="<h2>5. Target Modelling Results</h2>")

#ERROR MESSAGES
no_error=widgets.HTML()
ta_error=widgets.HTML(
    value="<p style='color:red;'>Type of Activity: Please select the \
      applicable option.</p>")
tc_error=widgets.HTML(
    value="<p style='color:red;'>Transport Category: Please select the \
      applicable option.</p>")
emissions_no_entry_error=widgets.HTML(
    value="<p style='color:red;'>Emissions must be provided.</p>")
emissions_unequal_error=widgets.HTML(
    value="<p style='color:red;'>Please ensure \
      (WTW emissions) = (WTT emissions) + (TTW emissions).</p>")
activity_error=widgets.HTML(
    value="<p style='color:red;'>Activity must be provided.</p>")
user_inputs_error=widgets.HTML(
    value="<p style='color:red;'>Information provided incomplete and/or \
      invalid. Please check and try again.</p>")

In [7]:
#WIDGET LAYOUT & STYLE

#SELECTION WIDGETS
#TC_WIDGET LAYOUT & STYLE CONTROLLED BY UPDATE_WIDGETS
layout={"width":"1100px"}
style={"description_width":"600px"}
ta_widget.layout=layout
ta_widget.style=style

#YEAR WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_widget.layout=layout
by_widget.style=style
ty_widget.layout=layout
ty_widget.style=style

#EMISSIONS WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
wtw_widget.layout=layout
wtw_widget.style=style
ttw_widget.layout=layout
ttw_widget.style=style
wtt_widget.layout=layout
wtt_widget.style=style

#ACTIVITY WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
bya_widget.layout=layout
bya_widget.style=style
tya_widget.layout=layout
tya_widget.style=style

#TEXT WIDGETS
layout={"width":"1100px"}
overall_header.layout=layout
basic_info_header.layout=layout
by_emissions_header.layout=layout
by_activity_header.layout=layout
ty_activity_header.layout=layout
ty_activity_footer.layout=layout
results_header.layout=layout
ta_error.layout=layout
emissions_no_entry_error.layout=layout
emissions_unequal_error.layout=layout
activity_error.layout=layout
user_inputs_error.layout=layout

# USER INPUT FORM (INTERACTIVITY)

In [8]:
#USER INPUT FORM - BASIC INFORMATION

def basic_info_check(ta,
                     tc,
                     by,
                     ty,
                     fc):
  basic_info_valid=True
  update_display(no_error,display_id="basic_info_check")
  #TA - USER MUST SELECT APPLICABLE OPTION
  if ta is None:
    basic_info_valid=False
    if fc:
      display(ta_error,display_id="basic_info_check")
  #TC - USER MUST SELECT APPLICABLE OPTION
  if ta is not None and tc is None:
    basic_info_valid=False
    if fc:
      display(tc_error,display_id="basic_info_check")
  return basic_info_valid
basic_info_input=interactive(
    basic_info_check,
    ta=ta_widget,
    tc=tc_widget,
    by=by_widget,
    ty=ty_widget,
    fc=fc_widget)

In [9]:
#USER INPUT FORM - BASE YEAR EMISSIONS

def by_emissions_check(wtw,
                       wtt,
                       ttw,
                       fc):
  by_emissions_valid=True
  update_display(no_error,display_id="by_emissions_check")
  #EITHER WTW, WTT, OR TTW MUST BE PROVIDED
  if wtw==0 and wtt==0 and ttw==0:
    by_emissions_valid=False
    if fc:
      display(emissions_no_entry_error,display_id="by_emissions_check")
  #WTT+TTW MUST EQUATE WTW
  if wtw!=wtt+ttw:
    if (wtw!=0 and wtt!=0 and ttw!=0) or \
     (wtw!=0 and wtt!=0 and ttw==0) or \
      (wtw!=0 and wtt==0 and ttw!=0) or \
       (wtw==0 and wtt!=0 and ttw!=0):
      by_emissions_valid=False
      if fc:
        display(emissions_unequal_error,display_id="by_emissions_check")
  return by_emissions_valid
by_emissions_input=interactive(
    by_emissions_check,
    wtw=wtw_widget,
    wtt=wtt_widget,
    ttw=ttw_widget,
    fc=fc_widget)

In [10]:
#USER INPUT FORM - BASE YEAR ACTIVITY

def by_activity_check(bya,
                      fc):
  by_activity_valid=True
  update_display(no_error,display_id="by_activity_check")
  #BYA - MUST BE PROVIDED
  if bya==0:
    by_activity_valid=False
    if fc:
      display(activity_error,display_id="by_activity_check")
  return by_activity_valid
by_activity_input=interactive(
    by_activity_check,
    bya=bya_widget,
    fc=fc_widget)

In [11]:
#USER INPUT FORM - TARGET YEAR ACTIVITY

def ty_activity_check(tya,
                      fc):
  ty_activity_valid=True
  update_display(no_error,display_id="ty_activity_check")
  #TYA - MUST BE PROVIDED
  if tya==0:
    ty_activity_valid=False
    if fc:
      display(activity_error,display_id="ty_activity_check")
  return ty_activity_valid
ty_activity_input=interactive(
    ty_activity_check,
    tya=tya_widget,
    fc=fc_widget)

# TARGET MODEL (DATA & PARAMETERS)

In [12]:
#IMPORT DATA

path="https://raw.githubusercontent.com/koklengyeo/migration/main/"

#IMPORT SECTOR DATA (B2DS)
b2ds=pd.read_csv(
    path+"transport_b2ds.csv",
    decimal=",",
    index_col=0,
    sep=";")
b2ds.columns=b2ds.columns.map(int)

In [13]:
#DEFINE PARAMETERS

pm={#INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    "iuc":1e3,
    #YEAR AXIS
    "year":pd.Series(range(2015,2051)).set_axis(range(2015,2051))}

# TARGET MODEL (FUNCTIONS)

In [14]:
#FUNCTION - CHECK IF USER INPUTS SATISFY IMPOSED CONDITIONS

def user_inputs_check():
  if not basic_info_input.result \
  or not by_emissions_input.result \
  or not by_activity_input.result \
  or not ty_activity_input.result:
    user_inputs_valid=False
  else:
    user_inputs_valid=True
  return user_inputs_valid

In [15]:
#FUNCTION - CAPTURE USER INPUTS INTO DICTIONARY

def user_inputs_capture():
  return {"ta":basic_info_input.children[0].value,
          "tc":basic_info_input.children[1].value,
          "by":basic_info_input.children[2].value,
          "ty":basic_info_input.children[3].value,
          "wtw":by_emissions_input.children[0].value,
          "wtt":by_emissions_input.children[1].value,
          "ttw":by_emissions_input.children[2].value,
          "bya":by_activity_input.children[0].value,
          "tya":ty_activity_input.children[0].value}

In [16]:
#FUNCTION - CONVERT USER INPUTS TO FORM REQUIRED BY MODEL

def user_inputs_convert(ip):
  #CONVERT EMISSIONS FROM TCO2E TO MTCO2E (I.E. SECTOR DATA UNIT)
  ip["wtw"]*=1e-6
  ip["wtt"]*=1e-6
  ip["ttw"]*=1e-6

  #CONVERT ACTIVITY FROM TKM & PKM TO BLN TKM & BLN PKM (I.E. SECTOR DATA UNIT)
  ip["bya"]*=1e-9
  ip["tya"]*=1e-9

  #ACTIVITY UNITS
  if ip["ta"]=="Freight":
    ip["au"]="tkm"
  elif ip["ta"]=="Passenger":
    ip["au"]="pkm"

  return ip

In [17]:
#FUNCTION - SECTOR DECARBONIZATION APPROACH

def sda(
    #SECTOR OR SECTOR SEGMENT NAME (IN SECTOR DATA)
    name=None,
    #BASE YEAR
    by=None,
    #TARGET YEAR
    ty=None,
    #BASE YEAR EMISSIONS (IN SAME UNIT AS SECTOR DATA)
    bye=None,
    #ACTIVITY FORECAST (IN SAME UNIT AS SECTOR DATA)
    af=None,
    #SECTOR DATA
    sector_data=None,
    #INTENSITY UNIT CONVERSION
    #INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    iuc=None):

  #SECTOR EMISSIONS, ACTIVITY & INTENSITY
  output=sector_data.loc[sector_data.index.str.contains(name)]
  for i in ["Emissions","Activity","Intensity"]:
    _=output.index[output.index.str.contains(name+" "+i)].values[0]
    output=output.rename(index={_:"Sector "+i})

  #COMPANY ACTIVITY
  output.loc["Company Activity"]=af

  #SDA (EQUATION 6 IN SDA REPORT)
  #SI TERM
  si=output.loc["Sector Intensity",2050]
  #P TERM
  p=output.loc["Sector Intensity"]-si
  p/=output.loc["Sector Intensity",by]-si
  #D TERM
  if output.loc["Company Activity",by]!=0:
    d=iuc*bye/output.loc["Company Activity",by]-si
  else:
    d=-si
  #M TERM
  m=output.loc["Company Activity",by]/output.loc["Sector Activity",by]
  m/=output.loc["Company Activity"]/output.loc["Sector Activity"]
  m[m>1]=1

  #COMPANY INTENSITY
  output.loc["Company Intensity"]=d*p*m+si
  output.loc["Company Intensity"]=output.loc["Company Intensity"].fillna(0)

  #COMPANY EMISSIONS
  output.loc["Company Emissions"]=\
  output.loc["Company Intensity"]*output.loc["Company Activity"]/iuc
  output.loc["Company Emissions"]=output.loc["Company Emissions"].fillna(0)

  #MASK YEAR<BASE YEAR
  output.loc[output.index.str.contains("Company"),output.columns<by]=np.nan

  #SORT
  output=output.sort_index()

  return output

In [18]:
#FUNCTION - TARGET MODEL

def target_model(
    #EMISSIONS CATEGORY (WTW, WTT OR TTW)
    ec=None):

  #ACTIVITY FORECAST
  af=ip["bya"]+(pm["year"]-ip["by"])*(ip["tya"]-ip["bya"])/(ip["ty"]-ip["by"])

  #NAME
  name=ip["ta"]+" "+ip["tc"]

  #SECTOR DATA
  _=name+" "+ec+"|"+name+" Activity"
  sector_data=b2ds.loc[b2ds.index.str.contains(_,regex=True)]
  for i in ["Emissions","Intensity"]:
    _=sector_data.index[sector_data.index.str.contains(i)].values[0]
    sector_data=sector_data.rename(index={_:name+" "+i})

  #SDA TARGET BASED ON B2DS
  output=sda(name=name,
             by=ip["by"],
             ty=ip["ty"],
             bye=ip[ec.lower()],
             af=af,
             sector_data=sector_data,
             iuc=pm["iuc"])

  #CONVERT EMISSIONS FROM MTCO2E TO TCO2E
  output.loc["Company Emissions"]*=1e6

  return output

def target_model_wrapper():
  return target_model(ec="WTW"),target_model(ec="WTT"),target_model(ec="TTW")

# RESULTS (FUNCTIONS)

In [19]:
#FUNCTION - SECTOR TABLE

def sector_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  _=ip["ta"]+" "+ip["tc"]+" Activity"
  table=b2ds.loc[b2ds.index.str.contains(_,regex=True)]
  table=table.loc[:,[ip["by"],ip["ty"]]]

  #% GROWTH
  _="% Growth ("+str(ip["by"])+"-"+str(ip["ty"])+")"
  table[_]=1e2*(table[ip["ty"]]/table[ip["by"]]-1)
  table=table[_].to_frame()

  #RENAME ROWS
  table.index=["Sector Activity ("+ip["au"]+")"]

  #ROUNDING
  table=table.apply(lambda x:round(x,2))

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(na_rep="",formatter="{0:.2f}%")
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  return table

In [20]:
#FUNCTION - TARGET TABLE

def target_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  table=[]
  if ip["wtw"]!=0:
    table.append(target_wtw.loc[["Company Emissions",
                                 "Company Intensity"],
                                 [ip["by"],ip["ty"]]])
    table[-1].index=["Company WTW Emissions (tCO2e)",
                     "Company WTW Intensity (gCO2e/"+ip["au"]+")"]
  if ip["wtt"]!=0:
    table.append(target_wtt.loc[["Company Emissions",
                                 "Company Intensity"],
                                 [ip["by"],ip["ty"]]])
    table[-1].index=["Company WTT Emissions (tCO2e)",
                     "Company WTT Intensity (gCO2e/"+ip["au"]+")"]
  if ip["ttw"]!=0:
    table.append(target_ttw.loc[["Company Emissions",
                                 "Company Intensity"],
                                 [ip["by"],ip["ty"]]])
    table[-1].index=["Company TTW Emissions (tCO2e)",
                     "Company TTW Intensity (gCO2e/"+ip["au"]+")"]
  if len(table)==1:
    table=table[0]
  elif len(table)>0:
    table=pd.concat(table)

  #% REDUCTION
  percentage_reduction_header=\
  "% Reduction ("+str(ip["by"])+"-"+str(ip["ty"])+")"
  table[percentage_reduction_header]=\
  -1e2*(table[ip["ty"]]/table[ip["by"]]-1)

  #RENAME COLUMNS
  table=table.rename({ip["by"]:"Base Year ("+str(ip["by"])+")",
                      ip["ty"]:"Target Year ("+str(ip["ty"])+")"},
                     axis=1)

  #ROUNDING
  table=table.apply(lambda x:round(x,2))

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(na_rep="",precision=2)
  table=table.format(na_rep="",formatter="{0:.2f}%",
                     subset=(slice(None),percentage_reduction_header))
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  return table

In [21]:
#FUNCTION USED IN COMPANY_TARGET_PLOT

#LABEL BASE YEAR & TARGET YEAR
def label_by_ty(series=None,ax=None):
  x=ip["by"]
  y=series[ip["by"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Base Year",color="blue",verticalalignment="bottom")
  x=ip["ty"]
  y=series[ip["ty"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Target Year",color="blue",verticalalignment="bottom")

In [22]:
#FUNCTION - COMPANY TARGET PLOT

def company_target_plot():
  #FETCH DATA (EXCLUDE YEAR<BASE YEAR)
  chart=target_wtw.iloc[:,target_wtw.columns>=ip["by"]]

  #SECTOR WTW EMISSIONS RELATIVE TO BASE YEAR
  rwtw=target_wtw.loc["Sector Emissions",target_wtw.columns>=ip["by"]]
  rwtw/=target_wtw.loc["Sector Emissions",ip["by"]]

  #SECTOR WTT EMISSIONS RELATIVE TO BASE YEAR
  rwtt=target_wtt.loc["Sector Emissions",target_wtt.columns>=ip["by"]]
  rwtt/=target_wtt.loc["Sector Emissions",ip["by"]]

  #SECTOR TTW EMISSIONS RELATIVE TO BASE YEAR
  rttw=target_ttw.loc["Sector Emissions",target_ttw.columns>=ip["by"]]
  rttw/=target_ttw.loc["Sector Emissions",ip["by"]]

  #SETUP FIGURE
  fig,(ax1,ax2)=plt.subplots(1,2,gridspec_kw={"wspace":0.3})
  fig.set_size_inches(10,4)

  #COMPANY & SECTOR INTENSITY PLOT
  ax1.plot(chart.loc["Sector Intensity"],
          color="orange",
          label="Sector WTW Intensity")
  ax1.plot(chart.loc["Company Intensity"],
            color="blue",
            label="Company WTW Intensity")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Intensity"],ax=ax1)
  #FORMAT
  ax1.grid()
  ax1.legend()
  ax1.set_title(ip["ta"]+" - "+ip["tc"]+"\nWTW Intensity Target")
  ax1.set_xlabel("Year")
  ax1.set_ylabel("Intensity (gCO2e/"+ip["au"]+")")

  #COMPANY & SECTOR EMISSIONS PLOT
  ax3=ax2.twinx()
  ax3.plot(rwtw,
          color="orange")
  ax2.plot([],color="orange",label="Sector WTW Emissions")
  if ip["tc"]!="Urban Rail":
    ax3.plot(rwtt,
            color="orange",
            linestyle="dotted")
    ax3.plot(rttw,
            color="orange",
            linestyle="dashed")
    ax2.plot([],color="orange",linestyle="dotted",label="Sector WTT Emissions")
    ax2.plot([],color="orange",linestyle="dashed",label="Sector TTW Emissions")
  ax2.plot(chart.loc["Company Emissions"],
          color="blue",
          label="Company WTW Emissions")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Emissions"],ax=ax2)
  #FORMAT
  ax2.grid()
  ax2.legend(loc="lower left")
  ax2.set_title(ip["ta"]+" - "+ip["tc"]+"\nWTW Emissions Target")
  ax2.set_xlabel("Year")
  ax2.set_ylabel("Company Emissions (tCO2e)")
  ax3.set_ylabel("Sector Emissions (Relative to Base Year)")

  plt.show(block=False)

# FINAL PRODUCT

In [23]:
#BUTTON WIDGET
calculate_target_button=widgets.Button(description="Calculate Target")

#OUTPUT WIDGET
results=widgets.Output()

#SPACER WIDGET
spacer=widgets.HTML(value="<br>")

#TARGET MODEL & RESULTS
@results.capture(clear_output=True)
def target_model_results(_):
  plt.close("all")
  results.outputs=[]
  #REGISTER FIRST CLICK
  fc_widget.value=True
  #USER INPUTS DO NOT MEET IMPOSED CONDITIONS
  if not user_inputs_check():
    #ERROR MESSAGE
    display(user_inputs_error)
  #USER INPUTS MEET IMPOSED CONDITIONS
  else:
    #TARGET MODEL
    global ip,target_wtw,target_wtt,target_ttw
    ip=user_inputs_convert(user_inputs_capture())
    target_wtw,target_wtt,target_ttw=target_model_wrapper()
    #RESULTS
    with results:
      display(results_header,
              spacer,
              sector_table(),
              spacer,
              target_table(),
              spacer)
      if ip["wtw"]!=0:
        company_target_plot()

#LINK BUTTON TO FUNCTION
calculate_target_button.on_click(target_model_results)

#DISPLAY
_=VBox([
    #USER INPUT FORM
    overall_header,
    basic_info_header,
    basic_info_input,
    by_emissions_header,
    by_emissions_input,
    by_activity_header,
    by_activity_input,
    ty_activity_header,
    ty_activity_input,
    ty_activity_footer,
    #TARGET MODEL & RESULTS
    calculate_target_button,
    results],
    layout={"width":"1110px"})
display(_)